In [6]:
%matplotlib inline

# For interactivity
# %matplotlib widget

# All imports
from random import choice
import numpy as np
import matplotlib.pyplot as plt

# Set number of decimal places to show
np.set_printoptions(formatter={'float': '{:.4f}'.format})

In [7]:
"""
Live updating:
use display.clear_output(wait=True)
Make an array for figures
Save figure by doing plt.plot, then append, then plt.close
Then to live update, display each figure using clear_output and then display.display()
use plt.clf() to clear figure
Maybe can stream instead of saving all figs?
"""

"""
Helpful functions: np.concatenate
Can choose axis to concat along
"""

'\nHelpful functions: np.concatenate\nCan choose axis to concat along\n'

In [8]:
def sigmoid(x: float) -> float:
    """Returns the sigmoid

    Args:
        x (float): the input

    Returns:
        float: the sigmoid of the input
    """
    return 1./(1. + np.exp(-x))

def sigmoid_prime(x: float) -> float:
    """Return the derivative of the sigmoid function

    Args:
        x (float): the input

    Returns:
        float: the derivative of the sigmoid of the input
    """
    g = sigmoid(x)
    return g*(1.-g)

def tanh(x: float) -> float:
    """Returns the hyperbolic tangent

    Args:
        x (float): input

    Returns:
        float: hyperbolic tangent of x
    """
    return ((np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x)))

def tanh_prime(x: float) -> float:
    """Returns the derivative of the hyperbolic tangent

    Args:
        x (float): input

    Returns:
        float: derivative of the hyperbolic tangent of x
    """
    g = tanh(x)
    return (1. - g**2)

In [ ]:
class NeuralNetwork:
    def __init__(self, layers: list, activation: str='sigmoid'):
        """Cosntructor for neural network

        Args:
            layers (list): number of inputs, hidden layers, and outputs as a list
            activation (str, optional): activation function to use. Only sigmoid and tanh are available. Defaults to 'signmoid'.
        """
        self.num_layers = len(layers)
        self.num_inputs = layers[0]
        self.num_outputs = layers[-1]
        self.num_hidden_layers = len(layers) - 2
        self.layers = layers
        print(f"num_inputs = {self.num_inputs}\nnum_outputs = {self.num_outputs}\nlayers = {self.layers}")
        
        # Create random weights
        self.weights = []
        for i in range(self.num_layers - 1):
            # Add bias weight for each neuron if not last layer
            if i < self.num_layers - 2:
                curr_weights = np.random.rand(layers[i] + 1, layers[i+1])
            else:
                curr_weights = np.random.rand(layers[i], layers[i+1])
            self.weights.append(curr_weights)

        # print(f"weights: {self.weights}")
        for weights in self.weights:
            print(f"weights: {weights.shape}")
        
        if activation == "sigmoid":
            self.activation = sigmoid
        elif activation == "tanh":
            self.activation = tanh
        else:
            raise Exception("Activation function must be 'sigmoid' or 'tanh'")

        self.activations = []
    
    def forward_prop(self, X, verbose=False):
        # Add bias
        # X = np.concatenate((X, np.ones(1))) # Add bias
        X = np.atleast_2d(X) # Convert to 2D matrix
        # post-activations array
        a_arr = [X]
        for layer, weight in enumerate(self.weights):
            # 1x4
            curr_layer_input = a_arr[layer]
            if layer < self.num_layers - 2: # Add bias upto layer preceding output layer
                curr_layer_input = np.concatenate((curr_layer_input, np.ones((1, 1))), axis=1)
            if verbose:
                print(f"Layer {layer}")
                print(curr_layer_input.shape)
                print(curr_layer_input[0].shape)
            # weight = 4x2
            dot_product = np.dot(curr_layer_input, weight)
            z_layer = np.atleast_2d(self.activation(dot_product))
            a_arr.append(z_layer)
        if verbose: 
            print(a_arr)
        self.activations = a_arr # Save activations at each layer
        return a_arr[0][0]
    
    def backward_prop():
        
        return

    def fit(self, X:list, y:list, learning_rate:float=0.2, steps:float=10**5, tolerance:float = 10**-2, verbose:bool = False):
        """_summary_

        Args:
            X (list): _description_
            y (list): _description_
            learning_rate (float, optional): _description_. Defaults to 0.2.
            steps (float, optional): _description_. Defaults to 10**5.
            tolerance (float, optional): _description_. Defaults to 10**-2.
            verbose (bool, optional): _description_. Defaults to False.
        """
        if not X or (len(X) != self.num_inputs):
            raise Exception("Number of inputs must match those specified in architecture")
        if not y or (self.num_outputs > 1 and len(y) != self.num_outputs):
            raise Exception("Number of outputs must match those specified in architecture")
        self.forward_prop(X)
        print(self.activations)
        return

    def find_RMS_error(self, X, y):
        """_summary_

        Args:
            X (_type_): _description_
            y (_type_): _description_
        """
        return

    def predict(self, x):
        """_summary_

        Args:
            x (_type_): _description_
        """
        return
    
    def visual_NN_boundaries(self, Nsamp=2000):
        """_summary_

        Args:
            Nsamp (int, optional): _description_. Defaults to 2000.
        """
        return
    

In [101]:
test = NeuralNetwork([3, 2, 3, 1])
test.fit([0.3, 0.4, 0.5], 0.5)
# test = NeuralNetwork([5, 2, 2, 3, 1])

num_inputs = 3
num_outputs = 1
layers = [3, 2, 3, 1]
weights: (4, 2)
weights: (3, 3)
weights: (3, 1)
[array([[0.3000, 0.4000, 0.5000]]), array([[0.7721, 0.6584]]), array([[0.6781, 0.6694, 0.7437]]), array([[0.7865]])]
